在Python中，通常使用quantile函数做等比分箱，使用cut函数做等宽分箱

In [1]:
import random
import pandas as pd

#按均匀分布生成100个介于10到100之间的实数
tmp = pd.Series([random.uniform(10, 100) for x in range(100)])

#1.使用pd.Series下的quantile函数进行等比分箱，此处将数据分成4份
x = tmp.quantile(q=[0.25,0.5,0.75, 1])
x.index = ['A','B','C','D']
tmp_quantile = tmp.apply(lambda m: x[x>=m].index[0]).values

#...另外常可通过均值、中位数、最大最小值来平滑数值以生成新的特征，这里拿均值来举例
y = tmp.groupby(tmp_quantile).mean()
tmp_qmean = [y[x] for x in tmp_quantile]

#2.使用cut函数进行等宽分箱，此处将数据分成5份
tmp_cut = pd.cut(tmp,bins=5,labels=["B1","B2","B3","B4","B5"])

#...另外可通过设置labels为NULL，并通过levles函数查看cut的水平
#...进一步确定各分箱的取值区间
#...可通过均值、中位数、最大最小值来平滑数值以生成新的特征，这里拿均值来举例
z = tmp.groupby(tmp_cut).mean()
tmp_cmean = [z[x] for x in tmp_cut]

使用Python基于iris数据集，以Species为目标变量，说明特征Sepal.Length的离散化过程

In [2]:
import pandas as pd
import numpy as np

iris = pd.read_csv("http://image.cador.cn/data/iris.csv")

def gains(u, v):
    ent_u = [np.sum([p*np.log2(1/p) for p in ct/np.sum(ct)]) for ct in [np.unique(u,return_counts=True)[1]]][0]
    v_id,v_ct = np.unique(v,return_counts=True)
    ent_u_m = [np.sum([p*np.log2(1/p) for p in ct/np.sum(ct)]) for ct in [np.unique(u[v==m],return_counts=True)[1] for m in v_id]]
    return ent_u - np.sum(np.array(ent_u_m)*(v_ct/np.sum(v_ct)))

def get_split_value(u,x):
    sorted_x, max_gains, e_split = np.sort(x), 0, min(x)
    for e in sorted_x:
        tmp = np.zeros(len(x))
        tmp[x>e]=1
        tmp_gain = gains(u,tmp)
        if tmp_gain > max_gains:
            max_gains,e_split  = tmp_gain, e
    return e_split

get_split_value(iris.Species,iris['Sepal.Length'].values)

5.5